In [1]:
import os 

from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from skimage import color
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from tensorflow import keras

### Load in grayscale data and labels

In [2]:
def clear():
    os.system('cls')

In [3]:
DATA_DIR = "../data/"
CIRCLES_DIR = os.path.join(DATA_DIR, "circles/")
TRIANGLES_DIR = os.path.join(DATA_DIR, "triangles/")
SQUARES_DIR = os.path.join(DATA_DIR, "squares/")

In [4]:
circles = os.listdir(CIRCLES_DIR)
triangles = os.listdir(TRIANGLES_DIR)
squares = os.listdir(SQUARES_DIR)

In [5]:
dataset = np.ndarray((300, 28, 28, 1))
for i in range(100):
    dataset[i, :, :, 0] = color.rgb2gray(
        plt.imread(os.path.join(CIRCLES_DIR, circles[i])))
    dataset[i + 100, :, :, 0] = color.rgb2gray(
        plt.imread(os.path.join(TRIANGLES_DIR, triangles[i])))
    dataset[i + 200, :, :, 0] = color.rgb2gray(
        plt.imread(os.path.join(SQUARES_DIR, squares[i])))

In [6]:
labels = np.concatenate((np.zeros(100), np.ones(100), np.zeros(100) + 2))
labels = keras.utils.to_categorical(labels)

### Train val split 

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    dataset, labels, test_size=0.2, stratify=labels, random_state=100)

### Optuna hyperparameter optimisation 

In [8]:
def create_model(depth, num_in_layer, pooling, num_per_depth, kernel, num_dense,
                 num_in_dense_layer, dropout, dropout_size, batch_norm):
    model = keras.models.Sequential()
    for i in range(depth):
        for j in range(num_per_depth):
            if i == 0 and j == 0:
                model.add(
                    keras.layers.Conv2D(
                        num_in_layer[i], (kernel[i], kernel[i]), 
                        activation="relu", input_shape=(28, 28, 1),
                        padding="same"))
            else:
                model.add(
                    keras.layers.Conv2D(
                        num_in_layer[i], (kernel[i], kernel[i]), 
                        activation="relu", padding="same"))
        if batch_norm:
            model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.MaxPooling2D(pooling[i]))
    model.add(keras.layers.Flatten())
    if dropout[0]:
        model.add(keras.layers.Dropout(dropout_size))
    model.add(keras.layers.Dense(num_in_dense_layer, activation="relu"))
    for i in range(num_dense):
        if dropout[i + 1]:
            model.add(keras.layers.Dropout(dropout_size))
        model.add(keras.layers.Dense(num_in_dense_layer, activation="relu"))
    model.add(keras.layers.Dense(3, activation="softmax"))
    return model

In [9]:
def objective(trial):
    lr = trial.suggest_categorical("lr", [1e-4, 1e-3, 2e-2, 1e-2])
    pooling_1 = trial.suggest_int("pooling_1", 2, 5)
    pooling_2 = trial.suggest_int("pooling_2", 2, 4)
    pooling_3 = trial.suggest_int("pooling_3", 2, 4)
    pooling_4 = trial.suggest_int("pooling_4", 2, 3)
    pooling = [pooling_1, pooling_2, pooling_3, pooling_4]
    
    kernel_1 = trial.suggest_int("kernel_1", 3, 6)
    kernel_2 = trial.suggest_int("kernel_2", 3, 4)
    kernel_3 = trial.suggest_int("kernel_3", 3, 4)
    kernel_4 = trial.suggest_int("kernel_4", 2, 3)
    kernel = [kernel_1, kernel_2, kernel_3, kernel_4]
    
    num_per_depth = trial.suggest_int("num_per_depth", 1, 3)
    
    depth = trial.suggest_int("depth", 1, 4) 
    num_in_layer = trial.suggest_categorical(
        "num_in_layer", [[4, 8, 16, 32], [16, 32, 64, 128], [32, 64, 128, 256]])
    num_in_layer = num_in_layer[:depth]
    
    num_dense = trial.suggest_int("num_dense", 0, 3)
    num_in_dense_layer = trial.suggest_categorical("num_in_dense_layer", [64, 96, 128, 256])
    
    dropout_size = trial.suggest_uniform("dropout", 0.1, 0.5)
    dropout_1 = trial.suggest_categorical("dropout_1", [True, False])
    dropout_2 = trial.suggest_categorical("dropout_2", [True, False])
    dropout_3 = trial.suggest_categorical("dropout_3", [True, False])
    dropout_4 = trial.suggest_categorical("dropout_4", [True, False])
    dropout = [dropout_1, dropout_2, dropout_3, dropout_4]
    
    batch_norm = trial.suggest_categorical("batch_norm", [True, False])
    
    try:
        model = create_model(depth, num_in_layer, pooling, num_per_depth, kernel, num_dense,
                             num_in_dense_layer, dropout, dropout_size, batch_norm)
    except:
        return 1e12  # If failed, return high loss in optuna 

    model.compile(
        loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=lr),
        metrics=["acc"])

    checkpoint = keras.callbacks.ModelCheckpoint(
        "optuna_model_{}.h5".format(trial.number), monitor="val_loss", mode="min")
    earlystop = keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min", patience=100, restore_best_weights=True)
    callbacks = [earlystop, checkpoint]
       
    model.summary()
    model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=1000, callbacks=callbacks)
    loss, acc = model.evaluate(x_val, y_val)
    return loss - acc  # maximize acc, minimize loss

In [10]:
# I removed the outputs from this as it's massive - but showcase the best trial below! 
study = optuna.create_study() 
# I got these below trials from a previous run - gave good results so I thought to reinclude them 
study.enqueue_trial({'lr': 0.01, 'pooling_1': 4, 'pooling_2': 3, 'pooling_3': 2, 'pooling_4': 2, 
                     'kernel_1': 4, 'kernel_2': 4, 'kernel_3': 4, 'kernel_4': 3, 'num_per_depth': 1, 
                     'depth': 2, 'num_in_layer': [16, 32, 64, 128], 'num_dense': 2, 
                     'num_in_dense_layer': 96, 'dropout': 0.19822203994283655, 'dropout_1': False, 
                     'dropout_2': True, 'dropout_3': True, 'dropout_4': True, 'batch_norm': True}) 
study.enqueue_trial({'lr': 0.01, 'pooling_1': 4, 'pooling_2': 2, 'pooling_3': 2, 'pooling_4': 2, 
                     'kernel_1': 4, 'kernel_2': 4, 'kernel_3': 4, 'kernel_4': 3, 'num_per_depth': 1, 
                     'depth': 2, 'num_in_layer': [32, 64, 128, 256], 'num_dense': 2, 
                     'num_in_dense_layer': 96, 'dropout': 0.19822203994283655, 'dropout_1': False, 
                     'dropout_2': True, 'dropout_3': True, 'dropout_4': True, 'batch_norm': True}) 
study.enqueue_trial({'lr': 0.02, 'pooling_1': 5, 'pooling_2': 2, 'pooling_3': 2, 'pooling_4': 2, 
                     'kernel_1': 3, 'kernel_2': 4, 'kernel_3': 4, 'kernel_4': 3, 'num_per_depth': 1, 
                     'depth': 3, 'num_in_layer': [4, 8, 16, 32], 'num_dense': 0, 
                     'num_in_dense_layer': 256, 'dropout': 0.13704670373767197, 'dropout_1': True, 
                     'dropout_2': True, 'dropout_3': False, 'dropout_4': False, 'batch_norm': True})
study.enqueue_trial({'lr': 0.01, 'pooling_1': 3, 'pooling_2': 2, 'pooling_3': 2, 'pooling_4': 2, 
                     'kernel_1': 4, 'kernel_2': 4, 'kernel_3': 4, 'kernel_4': 3, 'num_per_depth': 1,
                     'depth': 3, 'num_in_layer': [32, 64, 128, 256], 'num_dense': 0, 
                     'num_in_dense_layer': 64, 'dropout': 0.14654681284707352, 'dropout_1': True, 
                     'dropout_2': True, 'dropout_3': False, 'dropout_4': False, 'batch_norm': True})
study.optimize(objective, n_trials=100) 
clear_output()

### Best trial evaluation

In [11]:
best_params = study.best_trial.params
print("Best Parameters:", best_params)

Best Parameters: {'lr': 0.001, 'pooling_1': 3, 'pooling_2': 3, 'pooling_3': 3, 'pooling_4': 3, 'kernel_1': 5, 'kernel_2': 3, 'kernel_3': 4, 'kernel_4': 2, 'num_per_depth': 2, 'depth': 3, 'num_in_layer': [32, 64, 128, 256], 'num_dense': 0, 'num_in_dense_layer': 128, 'dropout': 0.22010822057383578, 'dropout_1': True, 'dropout_2': False, 'dropout_3': False, 'dropout_4': False, 'batch_norm': True}


In [12]:
best_number = study.best_trial.number
print("Best Number:", best_number)

Best Number: 93


In [13]:
model = keras.models.load_model("optuna_model_{}.h5".format(best_number))

In [14]:
model.evaluate(x_train, y_train)

8/8 [==============================] - 0s 4ms/step - loss: 3.3108e-06 - acc: 1.0000


[3.3108458410424646e-06, 1.0]

In [15]:
model.evaluate(x_val, y_val)

2/2 [==============================] - 0s 5ms/step - loss: 7.8272e-05 - acc: 1.0000


[7.827176887076348e-05, 1.0]

In [16]:
def class_report(model, x, y):
    predicted = np.argmax(model.predict(x), axis=-1)
    true = np.argmax(y, axis=-1)
    cm = classification_report(true, predicted)
    print(cm)
    return true, predicted

def show_misclass(x, true, predicted):
    misclass = np.where(true != predicted)[0]
    for i in misclass:
        print("True:", true[i])
        print("Predicted:", predicted[i])
        plt.imshow(x[i, :, :, 0])
        plt.show()  

In [17]:
true, predicted = class_report(model, x_train, y_train)
show_misclass(x_train, true, predicted)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00        80
           2       1.00      1.00      1.00        80

    accuracy                           1.00       240
   macro avg       1.00      1.00      1.00       240
weighted avg       1.00      1.00      1.00       240



In [18]:
true, predicted = class_report(model, x_val, y_val)
show_misclass(x_val, true, predicted)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



### K-fold cross validation on this architecture

In [19]:
def recreate_from_params(params):
    pooling = [params["pooling_1"], params["pooling_2"], params["pooling_3"], params["pooling_4"]]
    kernel = [params["kernel_1"], params["kernel_2"], params["kernel_3"], params["kernel_4"]]
    dropout = [params["dropout_1"], params["dropout_2"], params["dropout_3"], params["dropout_4"]]
    num_per_depth = params["num_per_depth"]
    depth = params["depth"]
    num_in_layer = params["num_in_layer"][:depth]
    num_dense = params["num_dense"]
    num_in_dense_layer = params["num_in_dense_layer"]
    dropout_size = params["dropout"]
    batch_norm = params["batch_norm"]

    model = create_model(depth, num_in_layer, pooling, num_per_depth, kernel, num_dense,
                         num_in_dense_layer, dropout, dropout_size, batch_norm)
    return model

In [20]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=42)

train_accuracies = np.zeros(n_splits)
val_accuracies = np.zeros(n_splits)

for i, (train_index, val_index) in enumerate(skf.split(dataset, np.argmax(labels, axis=1))):
    model = recreate_from_params(best_params)
    model.compile(
        loss="categorical_crossentropy", 
        optimizer=keras.optimizers.Adam(lr=best_params["lr"]),
        metrics=["acc"])
    
    earlystop = keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min", patience=100, restore_best_weights=True)
    callbacks = [earlystop]
    
    model.fit(dataset[train_index], labels[train_index], 
              validation_data=(dataset[val_index], labels[val_index]),
              epochs=1000, callbacks=callbacks, verbose=2)
    train_accuracies[i] = model.evaluate(dataset[train_index], labels[train_index])[1]
    val_accuracies[i] = model.evaluate(dataset[val_index], labels[val_index])[1]

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Epoch 1/1000
8/8 - 1s - loss: 1.6254 - acc: 0.3417 - val_loss: 1.0981 - val_acc: 0.3333
Epoch 2/1000
8/8 - 0s - loss: 1.0190 - acc: 0.5750 - val_loss: 1.0975 - val_acc: 0.3333
Epoch 3/1000
8/8 - 0s - loss: 0.6316 - acc: 0.7458 - val_loss: 1.0977 - val_acc: 0.3333
Epoch 4/1000
8/8 - 0s - loss: 0.5540 - acc: 0.7792 - val_loss: 1.1027 - val_acc: 0.3333
Epoch 5/1000
8/8 - 0s - loss: 0.3929 - acc: 0.8500 - val_loss: 1.1277 - val_acc: 0.3333
Epoch 6/1000
8/8 - 0s - loss: 0.2460 - acc: 0.8833 - val_loss: 1.1723 - val_acc: 0.3333
Epoch 7/1000
8/8 - 0s - loss: 0.2166 - acc: 0.9083 - val_loss: 1.2556 - val_acc: 0.3333
Epoch 8/1000
8/8 - 0s - loss: 0.0865 - acc: 0.9792 - val_loss: 1.3421 - val_acc: 0.3333
Epoch 9/1000
8/8 - 0s - loss: 0.0543 - acc: 0.9833 - val_loss: 1.3767 - val_acc: 0.3333
Epoch 10/1000
8/8 - 0s - loss: 0.0393 - acc: 0.9917 - val_loss: 1.3967 - val_acc: 0.3333
Epoch 11/1000
8/8 - 0s - loss: 0.0667 - acc: 0.9875 - val_loss: 1.3852 - val_acc: 0.3333
Epoch 12/1000
8/8 - 0s - loss:

In [21]:
print("KFold CV Val Scores:")
print("All:", val_accuracies)
print("Mean:", np.round(np.mean(val_accuracies), 2))
print("Min:", np.round(np.min(val_accuracies), 2))
print("Max:", np.round(np.max(val_accuracies), 2))

KFold CV Val Scores:
All: [0.96666664 1.         0.94999999 1.         0.93333334]
Mean: 0.97
Min: 0.93
Max: 1.0


In [22]:
print("KFold CV Val Scores:")
print("All:", train_accuracies)
print("Mean:", np.round(np.mean(train_accuracies), 2))
print("Min:", np.round(np.min(train_accuracies), 2))
print("Max:", np.round(np.max(train_accuracies), 2))

KFold CV Val Scores:
All: [1.         1.         1.         1.         0.99583334]
Mean: 1.0
Min: 1.0
Max: 1.0


In [23]:
# So not quite 100% accuracy as with the initial split, but not bad! 
# Would need a final test set for evaluation of generalisability 